# Preprocessing data

### Note: All the data we will work with in this workshop is just a sample of data that you will be given during the event itself!

## Some package imports

In [1]:
!pip install -U datalearn19intro
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datalearn19intro import get_accounts, get_events, get_subscriptions, get_users
%matplotlib inline

Looking in indexes: https://pypi.org/simple, https://pypi.org/simple
^C
Operation cancelled by user


## Reading the data

In [ ]:
accounts = get_accounts()
users = get_users()
events = get_events()
subscriptions = get_subscriptions()

In [ ]:
df = \
         pd.merge(pd.merge(accounts, users, on = 'account_id', how = 'outer', suffixes = ('_fromaccounts', '_fromusers')),
         pd.merge(events, subscriptions, on = 'account_id', how = 'outer', suffixes = ('_fromevents', '_fromsubscriptions')),
         how = 'outer', on = ['user_id', 'account_id'])

In [ ]:
df.shape

In [ ]:
n_rows = len(df)
n_cols = len(df.columns)

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 200)

In [ ]:
df.head(10)

In [ ]:
# joined_data.nunique()

**Notice:** Because of the join operation we now have much more rows than accounts, which are the actual entries in our dataset. We will see how to get back to one-row-per-account soon.

# Imputation

## Dropping featues with too much missing data

`dropna()` is an important `pandas` method. Use `axis=1` to drop columns and `axis=0`; `how` can be used to drop a row or column if it has `any` or `all` missing values. Use `thresh` to define an integer threshold for **non-missing** values instead.

In [ ]:
import math

In [ ]:
AT_LEAST_NON_MISSING_PERCENT = 0.7  # require at most 30% missing values per feature

In [ ]:
AT_LEAST_NON_MISSING = math.floor(AT_LEAST_NON_MISSING_PERCENT * len(df))
AT_LEAST_NON_MISSING

In [ ]:
df.dropna(axis=1, thresh=AT_LEAST_NON_MISSING, inplace=True)

In [ ]:
len(df.columns)

In [ ]:
print("{} columns dropped.".format(n_cols - len(df.columns)))

We went from 108 to 64 features!

## Dropping rows with too much missing data

In [ ]:
AT_LEAST_NON_MISSING_PERCENT_ROWS = 0.5  # require at most 50% missing values per row

In [ ]:
AT_LEAST_NON_MISSING_ROW = math.floor(AT_LEAST_NON_MISSING_PERCENT_ROWS * len(df.columns))
AT_LEAST_NON_MISSING_ROW

In [ ]:
df.dropna(axis=0, thresh=AT_LEAST_NON_MISSING_ROW, inplace=True)

In [ ]:
len(df)

In [ ]:
print("{} rows dropped.".format(n_rows - len(df)))

## Dummy "missing" flag

In [ ]:
df.nunique()[df.nunique() < 11]

In [ ]:
df.browser_fromaccounts.value_counts()

We won't see missing value counts like this. We need to add an argument:

In [ ]:
df.browser_fromaccounts.value_counts(dropna=False)

One option is to put a special flag for missing:

In [ ]:
filled_browser = df.browser_fromaccounts.fillna('missing')

In [ ]:
filled_browser.value_counts()

But in this case, just putting this into the `generic browser` category might not be a bad idea.

In [ ]:
filled_browser = df.browser_fromaccounts.fillna('generic browser')

In [ ]:
filled_browser.value_counts()

Let's just go with `missing` for this one, though...

In [ ]:
df.browser_fromaccounts = df.browser_fromaccounts.fillna('missing')

In [ ]:
df.browser_fromaccounts.value_counts()

## Filling missing value with the mean/median/mode...

Let's find a numerical feature with enough missing data...

In [ ]:
df.columns[(df.nunique() > 50) & (df.isna().sum() > 50)]

In [ ]:
df.raw_events.hist(bins=40)

In [ ]:
df.raw_events[df.raw_events<40].hist(bins=40)

In [ ]:
df.raw_events.isna().sum()

In [ ]:
df.raw_events.mean()

This is an integer feature, though! No sense in putting a real number as a stand-in. Let's look at the mode and median instead. Recall these are outlier-robust and skewness-robust statistics, and the distribution above seems very skewed...

In [ ]:
df.raw_events.median()

In [ ]:
df.raw_events.mode()

Filling missing values with zeros makes **a lot** of sense, in this case.

In [ ]:
df.raw_events.fillna(value=0, inplace=True)

In [ ]:
df.raw_events.isna().sum()

In [ ]:
df.raw_events[df.raw_events<40].hist(bins=40)

#### Back to the presentation! We now discuss handling outliers...

# Handling outliers

## Feature-wise outliers

In [ ]:
tdif = df.time_diff_fromaccounts.dropna()

In [ ]:
sns.distplot(tdif)

Doesn't look very normal...

In [ ]:
from scipy.stats import norm

In [ ]:
mu, std = norm.fit(tdif)
# Plot the histogram.
plt.hist(tdif, bins=25, density=True, alpha=0.6, color='g')
# Plot the PDF.
xmin, xmax = plt.xlim()
x = np.linspace(xmin, xmax, 100)
p = norm.pdf(x, mu, std)
plt.plot(x, p, 'k', linewidth=2)
title = "Fit results: mu = %.2f,  std = %.2f" % (mu, std)
plt.title(title)
# plot outlier boundaries
plt.axvline(x=std*3)
plt.axvline(x=-std*3)
plt.show()

This still doesn't look normal at all, so we won't use the three-standard-deviation rule for outlier removal in this case. In less obvious cases, it is better to use a normality test.

In [ ]:
from IPython.display import Image
Image(url="https://upload.wikimedia.org/wikipedia/commons/3/3a/Standard_deviation_diagram_micro.svg")

Let's look at another one...

In [ ]:
df.total_events.hist(bins=40)

This is a feature with a very skewed, long-tailed distribution. Forgoing scaling for now, we can mitigate the effect of outliers in this feature by trimming it to some max value. In this case 1000 or 500 looks reasonable.

In [ ]:
df.total_events = df.total_events.apply(lambda x: x if x <= 500 else 500)

In [ ]:
df.total_events.hist(bins=40)

## Overall outliers

There are many ways to detect outlies in your data. We will demonstrate just one.

But first, let's project our data to two dimensions, for each of presentation. We'll use a common method to do just that, named t-SNE, **without explaining it at all**:

In [ ]:
import time
from sklearn.manifold import TSNE

In [ ]:
ndf = df.select_dtypes('number')
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(ndf.fillna(0).values)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))

In [ ]:
tsne = df[['account_id']].copy(deep=True)
tsne['tsne-2d-one'] = tsne_results[:,0]
tsne['tsne-2d-two'] = tsne_results[:,1]

In [ ]:
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    palette=sns.color_palette("hls", 10),
    data=tsne,
    legend="full",
    alpha=0.3
)

We had to use only numeric data and very stupidly fill all NaNs with zero just to use this, but you should really do this **AFTER** most if not all of your feature engineering pipeline - especially scaling - so all your data is numeric, and possibly also NaN-less (if you're using a method that doesn't handle NaN input).

In [ ]:
len(ndf)

We will use a method called *isolation forest*, again with **absolutely no explantion**, because we simply don't have the time. [Read more about it here](https://towardsdatascience.com/outlier-detection-with-isolation-forest-3d190448d45e).

In [ ]:
from sklearn.ensemble import IsolationForest
clf = IsolationForest(max_samples=100000, random_state=42)
y_pred = clf.fit_predict(ndf.fillna(0))

In [ ]:
tsne['outlier'] = y_pred

In [ ]:
np.unique(y_pred, return_counts=True)

In [ ]:
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    hue="outlier",
    palette=sns.color_palette("hls", 2),
    data=tsne,
    legend="full",
    alpha=0.3
)

We can now drop outliers by utlizing the fact that our helper dataframe, `tsne`, has the same index as our dataset dataframe, since it's started as a copy of it; this would have worked whatever index we had, and does not rely on the fact the index is simply 0...11438.

In [ ]:
inlier_indices = tsne[tsne.outlier == 1].index
len(inlier_indices)

In [ ]:
# This line would throw away all outliers; feel free to uncomment it
# df = df.loc[inlier_indices]

### Note: In the context of training prediction models, it is important to remove outliers only in your training set, and not the subset kept for testing/estimating model performance!

#### Back to the presentation! We now discuss scaling and normalization...

# Scaling and normalization

Let's find a numeric feature to scale.

In [ ]:
ndf = df.select_dtypes('number')

In [ ]:
ndf.head(4)

In [ ]:
ndf.describe()

In [ ]:
ndf.max_team_size.hist(bins=40)

In [ ]:
ndf.max_team_size.value_counts(dropna=False)

In [ ]:
df.max_team_size.fillna(5, inplace=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
minmaxer = MinMaxScaler()

In [ ]:
df['max_team_size_minmaxed'] = df.max_team_size
df['max_team_size_minmaxed'] = minmaxer.fit_transform(df[['max_team_size']])

In [ ]:
df['max_team_size_minmaxed'].hist()

We can alternatively log transform this feature.

In [ ]:
df['log_max_tsize'] = df.max_team_size + 0.01  # push zeros a bit up

In [ ]:
df.log_max_tsize = np.log(df.log_max_tsize)

In [ ]:
sns.distplot(df.log_max_tsize)

# Feature extraction

## One-hot encoding

Let's find a nice categorical feature to one-hot encode...

In [ ]:
odf = df.select_dtypes('object')

In [ ]:
odf.shape

In [ ]:
odf.sample(5)

In [ ]:
odf.os_fromusers.value_counts(dropna=False)

Perfect for one-hot encoding!

When we one-hot encode categorical features it is very important to drop one of the features, to avoid [perfect multicolinearity (aka *the dummy variable trap*)](https://www.algosome.com/articles/dummy-variable-trap-regression.html). Since we have `NaNs` in our data, and we do not use the `dummy_na=True` option, a column for them will not be created, and we have effectively dropped the `os_NaN` column. When that is not the case, use `drop_first=True`.

In [ ]:
pd.get_dummies(df[['os_fromusers']], prefix='os').head(8)

In [ ]:
df = pd.concat([df, pd.get_dummies(df[['os_fromusers']], prefix='os')], axis=1)

Remember to drop the original feature!

In [ ]:
df.drop('os_fromusers', axis=1, inplace=True)

In [ ]:
df.head(5)

## Binning

This is a feature we should leave continous, but just for demonstation, let's assume it makes sense to bin it...
Maybe it would have made sense to bin it into zero and non-zero if we didn't have the `paying` boolean feature.

In [ ]:
df.collection_21_days.hist(bins=40)

In [ ]:
df.collection_21_days = df.collection_21_days.fillna(0)

In [ ]:
df.collection_21_days.hist(bins=40)

In [ ]:
df.collection_binned = pd.cut(
    x=df.collection_21_days,
    bins=[-1,0.1,200, 800, 900000],
    labels=['0','<200','200-800','>800'],
)

In [ ]:
df.collection_binned.value_counts().plot(kind='bar')

We know have a feature which we can one-hot encode to work with!

# Grouping

Let's group our data on `account_id` to get back to one row per account.

In [ ]:
grouped = df.groupby('account_id')

In [ ]:
df.head(20)

We can now use this object to start building some of the features.

Per-account features we can just reduce by taking the first value...

In [ ]:
per_account_features = [
    'is_gmail_fromaccounts', 'has_logo', 'collection_21_days', 'billed_users_count',
    'log_max_tsize', 'lead_score']

In [ ]:
firsts = grouped[per_account_features].first()

Numerical features that might be different per user, event or subscription, we need to summarize in an appropriate way.

For example, our dummy os variables come from the user table, so taking their mean when reducing will simply give us the ratio of users using each os in the organization. The same is true for other per-user features.

In [ ]:
mean_features = [
    'os_android', 'os_chrome_os', 'os_ios', 'os_linux', 'os_mac', 'os_windows',
    'has_photo', 'gender', 'has_phone'
]

In [ ]:
means = grouped[mean_features].mean().add_suffix('_avg')

In the case of events the total number might interest us instead:

In [ ]:
sum_features = [
    'total_events', 'notification_events', 'new_entry_events', 'payment_events',
    'inbox_events', 'communicating_events', 'non_communicating_events', 'web_events',
    'ios_events', 'android_events', 'desktop_app_events', 'empty_events']

In [ ]:
sums = grouped[sum_features].sum().add_suffix('_sum')

In [ ]:
new_df = pd.concat([firsts, means, sums], axis=1)

In [ ]:
new_df.head(10)

Now let's scale all features.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
minmaxer = MinMaxScaler()

In [ ]:
new_df[new_df.columns] = minmaxer.fit_transform(new_df)

In [ ]:
new_df.shape

In [ ]:
new_df.head(10)

That's it for some very basic feature extraction! You'll probably see basic ways to handle time data in our more in-depth preprocessing workshop on the first night of DataHack 2019!

# Dimensionality reduction

Let's see if the labels are somewhat separated in the resulting feature space.

In [ ]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(new_df.values)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))
tsne = new_df[['lead_score']].copy(deep=True)
tsne['tsne-2d-one'] = tsne_results[:,0]
tsne['tsne-2d-two'] = tsne_results[:,1]
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    palette=sns.color_palette("hls", 2),
    hue='lead_score',
    data=tsne,
    legend="full",
    alpha=1,
)

### Not bad!

# Feature selection

Select the best 20 features by chi-squared stats between each feature and labels.

In [ ]:
X = new_df.drop('lead_score', axis=1)
y = new_df[['lead_score']]

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
selector = SelectKBest(chi2, k=20)
selector.fit(X, y)
col_indices = selector.get_support(indices=True)
# Create new dataframe with only desired columns, or overwrite existing
selected_X = X[X.columns[col_indices]]
selected_df = pd.concat([selected_X, y], axis=1)

In [ ]:
time_start = time.time()
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(selected_df.values)
print('t-SNE done! Time elapsed: {} seconds'.format(time.time()-time_start))
tsne = new_df[['lead_score']].copy(deep=True)
tsne['tsne-2d-one'] = tsne_results[:,0]
tsne['tsne-2d-two'] = tsne_results[:,1]
plt.figure(figsize=(16,10))
sns.scatterplot(
    x="tsne-2d-one", y="tsne-2d-two",
    palette=sns.color_palette("hls", 2),
    hue='lead_score',
    data=tsne,
    legend="full",
    alpha=1,
)

### Let's save what we've done so we can use it to train our machine learning models!

In [ ]:
# selected_df.to_csv('data/monday_datalearn.csv')